# Simon's Algorithm
### Authors: Shane Barratt, Yash Shah, Julien Bloch

## Introduction

You've probably heard of Quantum Computing, either in the news or from your friends. But what's all the hype? Can a quantum computer really outperform a classical computer? Simon says it can.

In computability theory, a Turing Machine, introduced by Alan Turing, is a theoretical machine used to examine the possibilities and limitations of computers. All modern silicon-based computers are "Turing-complete", in the sense that they can perform any computation that a Turing Machine can. The theory is further extended to something called a Probabilstic Turing Machine, a non-deterministic Turing Machine which chooses between state transitions based on a probability distribution.

Simon, in his seminal 1994 paper "On the Power of Quantum Computation, introduced a problem and quantum algorithm which provides compelling evidence for a computational advantage of the quantum model of computation over probabilistic Turing machines.

This Jupyter Notebook seeks to provide a simulation of his algorithm, as well as interactive widgets and plots for you to test out different parameters.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
from functools import reduce
%matplotlib inline

//anaconda/envs/tensorflow/lib/python3.5/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


The basic setup of Simon's algorithm is that you're given a function $f: \{ 0,1 \}^n \to \{ 0,1 \}^m$, with $m \geq n$ and either 

a) f is 1-to-1   
or   
b) there exists an $s$ such that $\forall x \neq x'$, $f(x) = f(x') \iff x' = x \oplus s$.




We want to determine which of these holds for $f$ and find $s$ in the second case. This problem seems impossible from the classical point of view; it will require enumeration of all possible $s$'s which would take time exponential in $n$.

We begin by defining multiple functions for use later.

In [85]:
def num_to_binary(x, n):
    # Returns x in n-bit binary representation
    st = ("{0:0%db}" % n).format(x)
    return np.array([int(s) for s in st])

def bitwise_dot_product(x, y, n):
    # Returns x.y (mod 2), where x and y are both n bits
    return np.sum(num_to_binary(np.bitwise_and(x, y), n)) % 2

def random_one_to_one_function(n, m):
    # Returns a random 1-1 f: {0,1}^n -> {0,1}^m
    assert m >= n
    xs = list(range(2**n))
    ys = list(range(2**m))
    random.shuffle(ys)
        
    def f(x):
        return ys[x]
    
    return f

def random_s_function(n, m, s):
    # Returns a random f: {0,1}^n -> {0,1}^m s.t. 
    assert m >= n
    assert s != 0
    
    xs = list(range(2**n))
    ys = list(range(2**m))
    random.shuffle(ys)
    
    for x in xs:
        ys[np.bitwise_xor(x, s)] = ys[x]
    
    def f(x):
        return ys[x]
    
    return f

Simon proposes an algorithm "Fourier-Twice" to solve this very problem. It is repeated here for your convenience:

Routine **Fourier-twice**
1. perform a Hadamard transformation on a string of $n$ zeros, producing $2^{-n/2}\sum_x |x>$
2. compute $f(x)$, concatenating the answer to $x$, thus producing $2^{-n/2} \sum_x |(x, f(x))>$
3. perform a Hadamard transformation on $x$, producing $2^{-n} \sum_y \sum_x (-1)^{x . y} |(y, f(x))>$

End **Fourier-twice**

The entire algorithm is to run **Fourier-twice** $n-1$ times, each time sampling the final superposition and noting the resulting $(y, f(x))$. Simon's algorithm is depicted pictorally below.

<img src="simons-diagram.png" alt="Drawing" style="width: 400px;"/>

In case a, where f is 1-1, the sum after step 3 of the algorithm will be a uniform superposition across all $(y, f(x))$ pairs. The algorithm will return $n-1$ i.i.d. $(y, f(x))$ pairs uniformly distributed amongst n-bit strings.

In case b, where f follows the structure described above (there exists an $s$ such that $\forall x \neq x'$, $f(x) = f(x') \iff x' = x \oplus s$), the sum after step 3 of the algorithm will be a uniform superposition across all $(y, f(x))$ s.t. $y.s=0$ because all terms where $y.s=1$ cancel out. We need $n-1$ independent equations and then we can solve for $s$ using Gaussian elimination.

This quantum algorithm requires $O(N)$ queries to the black box whereas a classical algorithm would require $O(2^{\frac{n}{2}})$ queries. Thus, Simon's conclusion is that quantum computing is inherently more powerful.

Since we are simulating quantum computation and at the same time arguing that it is inherently more powerful, we will keep $n=10$ so that the computations are feasible.

In [47]:
n = 10
m = n

In [48]:
possible_ys = []
for y in range(2**n):
    if bitwise_dot_product(y, s, n) == 0:
        possible_ys.append(y)

def fourier_twice(f, n, s, onetoone=False):
    x = random.randint(0, n-1)
    y = random.randint(0, n-1)
    if not onetoone:
        y = random.choice(possible_ys)
    return y, f(x)

Successful operation of Simon's algorithm in the case where there is a $s$ for $f$ requires that the $n-1$ equations generated are linearly independent so that we can solve for $s$. We present here a simulation of this very condition.

In [49]:
import itertools
masks = list(itertools.product([0,1], repeat=n-2)) # all n-2 bitstring masks (for linear independence calculation)

In [55]:
def sim_lin_indep():
    s = np.random.randint(1, 2**n)
    f = random_s_function(n, m, s)
    outputs = [fourier_twice(f, n, s) for _ in range(n-1)]
    y_outputs = [o[0] for o in outputs]
    for i in range(len(outputs)):
        for mask in masks:
            res = reduce(np.bitwise_xor, np.array(y_outputs[:i] + y_outputs[i+1:])*np.array(mask))
            if res == y_outputs[i]:
                return False
    return True

In [56]:
%timeit sim_lin_indep()

100 loops, best of 3: 17.4 ms per loop


In [57]:
lin_indep_count = 0.0
for _ in range(1000):
    lin_indep_count += 1 if sim_lin_indep() else 0

In [58]:
print ("prob linear independent:", 100*lin_indep_count/1000, "%")

prob linear independent: 27.1 %


So it seems like the probability of getting $n-1$ linearly independent equations and to be able to solve the system of equations is about $\frac{1}{4}$. We can treat every run of the algorithm as independent and thus the success of this algorithm becomes a realization of a [Bernoulli Process](wikipedia bernoulli process), which has an expected number of trials for interarrival times of $1/p=1/\frac{1}{4}=4$ and variance = $\frac{1-p}{p}=\frac{.75}{.25}=3$. Since $n$ is small, we solve the system of equations by brute force.

In [86]:
for _ in range(20):# print solutions
    s = np.random.randint(1, 2**n)
    f = random_s_function(n, m, s)
    outputs = [fourier_twice(f, n, s) for _ in range(n-1)]
    final_s = []
    for possible_s in range(2**n):
        good = True
        for y, f_x in outputs:
            good = good and (np.sum(num_to_binary(np.bitwise_and(y, possible_s), n)) % 2 == 0)
        if good:
            final_s.append(possible_s)

    print (final_s)

[0, 432, 693, 773]
[0, 693]
[0, 221, 305, 492, 616, 693, 857, 900]
[0, 93, 693, 744]
[0, 372, 693, 961]
[0, 185, 524, 693]
[0, 390, 693, 819]
[0, 382, 693, 971]
[0, 693]
[0, 693]
[0, 693]
[0, 112, 437, 453, 693, 709, 768, 880]
[0, 693]
[0, 105, 693, 732]
[0, 171, 309, 414, 542, 693, 811, 896]
[0, 106, 693, 735]
[0, 222, 619, 693]
[0, 693]
[0, 693]
[0, 466, 693, 871]


In [87]:
f(s) == f(0)

True

As expected, $0$ is always a solution. Every time, $s$ is in the list of solutions but sometimes there are other possible solutions because the equations are dependent.\

Now we will examine the case where $f$ is a 1-1 function and see if our quantum algorithm can discriminate it.

In [65]:
n = 10
m = n

In [69]:
f = random_one_to_one_function(n, m)

In [72]:
for _ in range(1):# print solutions
    outputs = [fourier_twice(f, n, s, onetoone=True) for _ in range(n-1)]
    final_s = []
    for possible_s in range(2**n):
        good = True
        for y, f_x in outputs:
            good = good and (np.sum(num_to_binary(np.bitwise_and(y, possible_s), n)) % 2 == 0)
        if good:
            final_s.append(possible_s)

    print (final_s)

[0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 256, 272, 288, 304, 320, 336, 352, 368, 384, 400, 416, 432, 448, 464, 480, 496, 512, 528, 544, 560, 576, 592, 608, 624, 640, 656, 672, 688, 704, 720, 736, 752, 768, 784, 800, 816, 832, 848, 864, 880, 896, 912, 928, 944, 960, 976, 992, 1008]


A simple evaluation of $f$ at $0$ and the $s$ found by the algorithm will determine if we have found the real $s$ or just a random string.

In [90]:
f(final_s[1]) == f(0)

False